In [1]:
import os # importing OS in order to make GPU visible
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]='1' 

In [2]:
import sys # importyng sys in order to access scripts located in a different folder

path2scripts = r'C:\Users\bcper\Documents\GitHub\models\research\\' 
sys.path.insert(0, path2scripts) # making scripts in models/research available for import

# Generate the TF .record files

In [ ]:
work_path = r'C:\Users\bcper\Documents\GitHub\xView_data_utilities'
image_path = r'C:\Users\bcper\Desktop\Dataset\train_images' + '\\'

#small image set for testing
#image_path = r'C:\Users\bcper\Desktop\images' + '\\'


geojson_name = r'\train_data_clean.geojson'
geojson_path = r'C:\Users\bcper\Desktop\Dataset' + geojson_name

os.chdir(work_path)

#name to be appended to the .record files
filename = 'Cleaned'

test_ratio = .2


%run process_wv.py $image_path $geojson_path -s=$filename -t=$test_ratio

# Create the label map

In [ ]:
#Filenames and locations
class_label_filename = 'Class Labels.txt'
class_label_path = r'C:\Users\bcper\Documents\GitHub\model_training' + '\\' + class_label_filename

label_map_name = 'label_map.pbtxt'
label_map_path = r'C:\Users\bcper\Desktop\Dataset' + '\\' + label_map_name



#read in the class labels
class_dict = {}
file = open(class_label_path, "r")

#create a class label dictionary
for line in file:
    key, value = line.split(':')
    class_dict[int(key)] = value.strip()


#this will make the list in the correct label format for tensorflow
x = ''
for key in class_dict:
    x = x + ('item { \n\tid: ' + str(key) + '\n\tname: \"' + class_dict.get(key) + '\"\n}\n\n')
    
# Write the label map to a file
#lmap = open(label_map_path,"w")#write mode
#lmap.write(x)
#lmap.close()

# Develop the .config file

In [52]:
def edit_pipeline(pipeline):

    # fine_tune_checkpoint
    pipeline = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(checkpoint), pipeline)
 
    # tfrecord files train and test.
    pipeline = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record), pipeline)
    pipeline = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record), pipeline)

    # label_map_path
    pipeline = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map), pipeline)

    # Set training batch_size.
    pipeline = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch), pipeline)

    # Set training steps, num_steps
    pipeline = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(steps), pipeline)
    
    # Set number of classes num_classes.
    pipeline = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(classes), pipeline)
    #f.write(s) 

In [41]:
#this should be the model directory of the cloned tensorflow git repo
wd = r'C:/Users/bcper/Documents/GitHub/models'
work_path = r'C:/Users/bcper/Documents/GitHub/models/workspace'
os.chdir(work_path)

model_name = 'ssd_resnet152_v1_fpn_640x640'
pretrained_model_dir = 'ssd_resnet152_v1_fpn_640x640_coco17_tpu-8'
model_version = 'v3'

config_source = r'/models/' + model_name + '/' + model_version + '/vbig/pipeline.config'
config_destination = r'/' + model_name + '/' + model_version + '/'

In [50]:
checkpoint

'C:\\Users\\bcper\\Documents\\GitHub\\models\\workspace\\pre-trained_models\\ssd_resnet152_v1_fpn_640x640\\ckpt-0'

In [53]:
import re
import io

classes = 60
steps = 10000
batch = 6

pipeline_inpath = r'C:\Users\bcper\Documents\GitHub\models\workspace\pre-trained_models' + '\\' + pretrained_model_dir + '\pipeline.config'
pipeline_outpath = r'C:\Users\bcper\Documents\GitHub\models\workspace\models' + '\\' + model_name + '\\' + model_version + '\pipeline.config'

data_dir_path = r'C:\Users\bcper\Documents\GitHub\models\workspace\data'

test_record = data_dir_path + r'\xview_test_Cleaned.record'
train_record = data_dir_path + r'\xview_train_Cleaned.record'
label_map = data_dir_path + r'\label_map.pbtxt'
checkpoint = r'C:\Users\bcper\Documents\GitHub\models\workspace\pre-trained_models' + '\\' + model_name + '\\ckpt-0'


with open(pipeline_inpath) as file:
    pipeline = file.read()
    
#print(pipeline)
     
if os.path.exists(pipeline_outpath):
    with open(pipeline_outpath, 'w') as f:
        print('Overwriting pipeline file')
        edit_pipeline(pipeline)
else:
    with open(pipeline_outpath, 'x') as f:
        print('Creating pipeline file')
        edit_pipeline(pipeline)

Overwriting pipeline file


error: bad escape \U at position 25

# Train the model

In [ ]:
#this should be the model directory of the cloned tensorflow git repo
wd = r'C:/Users/bcper/Documents/GitHub/models'
work_path = r'C:/Users/bcper/Documents/GitHub/models/workspace'

os.chdir(work_path)

model_name = 'ssd_resnet152_v1_fpn_640x640'
model_version = 'v2'



config_path = r'/models/' + model_name + '/' + model_version + '/vbig/pipeline.config'
model_path = r'/' + model_name + '/' + model_version + '/'
config_path = work_path + r'/models/'+ model_name + '/' + model_version + '/pipeline.config'
model_path = work_path + r'/models/'+ model_name + '/' + model_version + '/' 
train_steps = 2000
eval_steps = 1


#run the model training
work_path = r'C:\Users\bcper\Documents\GitHub\models\workspace'
os.chdir(work_path)

#this fails to train for some reason
#!python model_main_tf2.py --pipeline_config_path=$config_path --model_dir=$model_path --alsologtostderr 
cmd_string = r'python model_main_tf2.py --pipeline_config_path=' + config_path + r' --model_dir=' + model_path + r' --alsologtostderr'

print(cmd_string)

# Start Tensorboard

In [ ]:
work_path = r'C:/Users/bcper/Documents/GitHub/models/workspace'
path = r'\models\resnet50'

os.chdir(work_path)

tensorboard_path = os.getcwd() + path


!tensorboard --logdir=$tensorboard_path

# Export Model

In [3]:
work_path = r'C:/Users/bcper/Documents/GitHub/models/workspace'
os.chdir(work_path)

model_name = 'resnet50_racoon'
model_version = 'v1'


pipeline_config_path = os.getcwd() + r'/models/'+ model_name + '/' + model_version + '/pipeline.config'
trained_checkpoint_dir=os.getcwd() + r'/models/' + model_name + '/' + model_version
output_directory=os.getcwd() + r'/exported_models/'+ model_name + '_' + model_version



%run exporter_main_v2.py --pipeline_config_path=$pipeline_config_path --trained_checkpoint_dir=$trained_checkpoint_dir --output_directory=$output_directory --input_type=image_tensor



Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))


W0607 15:54:37.384699 10592 deprecation.py:596] From C:\Users\bcper\anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py:463: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))


INFO:tensorflow:depth of additional conv before box predictor: 0


I0607 15:54:40.814950 10592 convolutional_keras_box_predictor.py:153] depth of additional conv before box predictor: 0


Instructions for updating:
Use ref() instead.


...

W0607 15:55:46.700710 10592 save.py:1239] FOR KERAS USERS: The object that you are saving contains one or more Keras models or layers. If you are loading the SavedModel with `tf.keras.models.load_model`, continue reading (otherwise, you may ignore the following instructions). Please change your code to save with `tf.keras.models.save_model` or `model.save`, and confirm that the file "keras.metadata" exists in the export directory. In the future, Keras will only load the SavedModels that have this file. In other words, `tf.saved_model.save` will no longer write SavedModels that can be recovered as Keras models (this will apply in TF 2.5).

FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: C:\Users\bcper\Documents\GitHub\models\workspace/exported_models/resnet50_racoon_v1\saved_model\assets


I0607 15:55:48.545793 10592 builder_impl.py:774] Assets written to: C:\Users\bcper\Documents\GitHub\models\workspace/exported_models/resnet50_racoon_v1\saved_model\assets


INFO:tensorflow:Writing pipeline config file to C:\Users\bcper\Documents\GitHub\models\workspace/exported_models/resnet50_racoon_v1\pipeline.config


I0607 15:55:54.708206 10592 config_util.py:253] Writing pipeline config file to C:\Users\bcper\Documents\GitHub\models\workspace/exported_models/resnet50_racoon_v1\pipeline.config
